In [4]:
# -*- coding: utf-8 -*-
"""
Created on Fri Oct 28 11:22:34 2016

@author: peyang

----
Fitness Selection function:
    1. rms ranking
    2. linear ranking
    3. tournament
    4. roulette wheel selection
----
Note:
    tagetSize: the target mate pool number
"""
import numpy as np
from numpy import random

def rms_ranking(rmses, singlepool, tagetSize):
    assert(len(singlepool) >= tagetSize)
    singleNum = len(singlepool)
    singlermses = [rmses[ii] for ii in singlepool]
    ascendingIndexes  = np.argsort(singlermses)
    return ascendingIndexes[:tagetSize]

def linear_ranking(rmses, singlepool, tagetSize, sp=1.5):
    '''
    Linear ranking selection without replacement.
    In rank-based fitness assignment, the population is sorted by fitness.
    Consider 'N' as the number of individuals in the population, 'Pos' the
    position of an individual in this population (least fit individual
    has Pos=0, the fittest individual Pos=N-1) and 'SP' the selective pressure.

    ----
    parameters:
        tagetSize:  int, target number for a full mate pool
        sp:         selection pressure, prob of selecting best individual
                    compared to the average selection prob

    ----
    fitness function:
        fitness = (2 - sp) + 2.*(sp-1)*Pos/(N-1)
    '''
    assert(len(singlepool) >= tagetSize)
    singleNum = len(singlepool)
    singlermses = [rmses[ii] for ii in singlepool]
    ascendingIndexes  = np.argsort(singlermses)
    descendingIndexes = ascendingIndexes[::-1]

    sp = 1.5 # hard-coded, 1<sp<2, selection pressure
    fitness = lambda x: 2 - sp + 2.*(sp-1)*x/(singleNum-1)

    props = [0. for ii in range(singleNum)]
    for ii, jj in enumerate(descendingIndexes):
        props[jj] = fitness(ii) # the probability in loc jj is by a portion of ii
    sumFitness = np.sum(props)
    props = [1.*x/sumFitness for x in props]

    draw = random.choice(singlepool, tagetSize, p=props, replace=False)
    return draw

def tournament(rmses, singlepool, tagetSize, tourSize = 2):
    '''
    Tournament selection without replacement.
    In tournament selection a number Tour of individuals is chosen randomly
    from the population and the best individual from this group is selected
    as parent. This process is repeated until the mate pool is full.

    Note
    ----
        singlepool: list, the indexes of current available single pool
        tagetSize:  int, target number for a full mate pool
        tourSize:   int, how many candidates are chosen into tournament
    '''
    assert(len(singlepool) >= tagetSize)
    if tagetSize == 0:
        return []
    if len(singlepool) < tourSize:
        return singlepool
    draw = random.choice(singlepool, tourSize, replace=False)
    drawRmses = [rmses[ii] for ii in draw]
    winneNo = np.argsort(drawRmses)[0]
    winner = draw[winneNo]

    # append winnerIdx into mate pool, and delete it from single pool
    tagetSize -= 1
    try:
        winnerIdx = singlepool.index(winner)
        del singlepool[winnerIdx]
    except ValueError:
        raise ValueError('The winner {} is not in pre-defined list: {}'.fromat(winner, singlepool))
    return [winner]+tournament(rmses, singlepool, tagetSize, tourSize)[:]

if __name__ == '__main__':
    random.seed(0)
    rmses = [round(random.random(), 3) for i in range(10)]
    singleNum = len(rmses)

    print 'rms ranking ...'
    for i in range(11):
        print rms_ranking(rmses, range(singleNum), i)

    print '\nlinear ranking ...'
    for i in range(11):
        random.seed(0)
        print linear_ranking(rmses, range(singleNum), i, 1.5)

    print '\ntournament ranking ...'
    for i in range(11):
        random.seed(0)
        print tournament(rmses, range(singleNum), i, 2)

rms ranking ...
[]
[9]
[9 4]
[9 4 6]
[9 4 6 3]
[9 4 6 3 0]
[9 4 6 3 0 2]
[9 4 6 3 0 2 5]
[9 4 6 3 0 2 5 1]
[9 4 6 3 0 2 5 1 7]
[9 4 6 3 0 2 5 1 7 8]

linear ranking ...
[]
[5]
[5 6]
[5 6 4]
[5 6 3 4]
[5 6 4 7 3]
[5 6 4 3 9 1]
[5 6 4 9 2 3 8]
[5 6 4 9 8 2 7 1]
[5 6 4 9 2 7 3 8 0]
[5 6 4 9 3 7 2 8 0 1]

tournament ranking ...
[]
[2]
[2, 4]
[2, 4, 3]
[2, 4, 3, 5]
[2, 4, 3, 5, 1]
[2, 4, 3, 5, 1, 9]
[2, 4, 3, 5, 1, 9, 7]
[2, 4, 3, 5, 1, 9, 7, 0]
[2, 4, 3, 5, 1, 9, 7, 0, 6]
[2, 4, 3, 5, 1, 9, 7, 0, 6, 8]
